# Error Analysis:
A common approach to testing the implementation of a numerical method is to compare the numerical solution with an exact analytical solution of the test problem and conclude that the program works if the error is “small enough”.
All we usually know about the numerical error is its asymptotic properties, for instance that it is proportional to $h^2$ if h is the size of a cell in the mesh. Then we can compare the error on meshes with different $h$-values to see if the asymptotic behavior is correct. However, if we have a test problem for which we know that there should be no approximation errors, we know that the analytical solution of the PDE problem should be reproduced to machine precision by the program.Typically, elements of degree $r$ can reproduce polynomials of degree $r$ exactly, so this is the starting point for constructing a solution without numerical approximation errors.
  
  
# Computing the error
Finally, we compute the error to check the accuracy of the solution. We do this by comparing the finite element solution u with the exact solution $u_e$, which in this example happens to be the same as the Expression used to set the boundary conditions. We compute the error in two different ways. First, we compute the $L^2$
norm of the error, defined by
$$E=\sqrt{\int_{\Omega}(u_e-u)dx }$$
Since the exact solution is quadratic and the finite element solution is piecewise linear, this error will be nonzero. The errornorm() function can also compute other error norms such as the $H^1$ norm.


# How to check that the error vanishes:
With inexact arithmetics, as we always have on a computer, the maximum error at the vertices is not zero, but should be a small number. The machine precision is about $10^{-16}$ , but in finite element calculations, rounding errors of this size may accumulate, to produce an error larger than $10^{-16}$. 
Experiments show that increasing the number of elements and increasing the degree of the finite element polynomials increases the error. For a mesh with $2×(20×20)$ cubic Lagrange elements (degree 3) the error is about $2×10^{-12}$, while for 81 linear elements the error is about $2×10^{-15}$.

# Computing convergence rates
A central question for any numerical method is its convergence rate: how fast does the error approach zero when the resolution is increased? For finite element methods, this typically corresponds to proving, theoretically or empirically, that the error $e=u_e−u$ is bounded by the mesh size $h$ to some power $r$; that is, $∥e∥≤Ch^r$ for some constant $C$. The number $r$ is called the convergence rate of the method. Note that different norms, like the $L2-norm ∥e∥$ or $H_0^1-norm ∥\nabla e∥$ typically have different convergence rates. A better value may be achieved by interpolating the exact solution into a higher-order function space, which can be done by simply increasing the degree of interpolation. To compute the error we use the function errornorm which computes the error norm in a more intelligent way. First, both $u_e$ and $u$ are interpolated into a higher-order function space. Then, the degrees of freedom of $u_e$ and $u$ are subtracted to produce a new function in the higher-order function space. Finally, so-called function integrates the square of the difference function to get the value of the error norm. Sometimes it is of interest to compute the error of the gradient field: $\parallel \nabla (u_e -u)\parallel$, often referred to as the $H_0^1$ or $H^1$ seminorm of the error.

E = errornorm($u_e$, u, normtype='$L^2$')

E = errornorm($u_e$, u, norm_type='$H_0^1$')


 

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import numpy as np 
import matplotlib.pyplot as plt

from solver import * 
from l2error import *
from h1error import *


start = 2
stop = 7
step = 1

v = 0
for k in range(start,stop,step):
    v = v + 1
    
    
h1_error = np.zeros(v)    
e2 = np.zeros(v)
h = np.zeros(v)
node_num = np.zeros(v)
elements = np.zeros(v)

print ( '' )
print ( '   Total Nodes    Total Elements           h                L2_error                H1_error' )
print ( '' )

v = 0
for k in range(start,stop,step):
    i = 2**k
    print ('i=',i)
    elements[v]= i * i
    u,h[v],node_num[v] = solver(element_linear_num = i)
    e2[v] = L2_error(element_linear_num = i,u = u)
    h1_error[v] = H1_error(element_linear_num = i,u = u)
    print ( '      %4d            %4d              %8f        %14g        %14g' 
           % ( node_num[v],elements[v], h[v], e2[v], h1_error[v] ) )
    v = v + 1 

print ( '' )

#
# plotting L2 error 
#
plt.plot(node_num,e2,'b')
plt.xlabel('Nodes')
plt.ylabel('$L_2 \,\,\, Error$')
plt.grid()
plt.title('Solution error in the $L_2$ norm\n with the total number of nodes')
plt.show()  


plt.plot(elements,e2,'b')
plt.xlabel('Elements')
plt.ylabel('$L_2 \,\,\, Error$')
plt.grid()
plt.title('Solution error in the $L_2$ norm\n with the total number of elements')
plt.show() 

plt.plot(h,e2,'r')
plt.xlabel('h')
plt.ylabel('$L_2 \,\,\, Error$')
plt.grid()
plt.title('Solution error in the $L_2$ norm\n with the grid spacing $\Delta x =  \Delta y = h $')
plt.show()  

plt.plot(np.log2(h),np.log2(e2),'r')
plt.xlabel('log(h)')
plt.ylabel('$log(L_2 \,\,\, Error)$')
plt.grid()
plt.show()  

#
# plotting H1 error 
#
plt.plot(node_num,h1_error,'b')
plt.xlabel('Nodes')
plt.ylabel('$H_1 \,\,\, Error$')
plt.grid()
plt.title('Solution error in the $H_1$ semi-norm\n with the total number of nodes')
plt.show()  


plt.plot(elements,h1_error,'b')
plt.xlabel('Elements')
plt.ylabel('$H_1 \,\,\, Error$')
plt.grid()
plt.title('Solution error in the $H_1$ semi-norm\n with the total number of elements')
plt.show() 

plt.plot(h,h1_error,'r')
plt.xlabel('h')
plt.ylabel('$H_1 \,\,\, Error$')
plt.grid()
plt.title('Solution error in the $H_1$ semi-norm\n with the grid spacing $\Delta x =  \Delta y = h $')
plt.show()  

plt.plot(np.log2(h),np.log2(h1_error),'r')
plt.xlabel('log(h)')
plt.ylabel('$log(H_1 \,\,\, Error)$')
plt.grid()
plt.show() 

#
# To compare the errors  
#
plt.plot(np.log2(h),np.log2(e2),'b', label = '$log(L_2 \,\,\, Error)$')
plt.plot(np.log2(h),np.log2(h1_error),'r',label = '$log(H_1 \,\,\, Error)$')
plt.xlabel('log(h)')
plt.legend()
plt.grid()
plt.show() 